In [ ]:
import os
import os.path as osp
root_dir = osp.dirname(osp.dirname(os.getcwd()))

import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
plt.rc('axes', axisbelow=True)
plt.rcParams.update({'figure.autolayout': True})

ram_dir = osp.join(root_dir, 'experiments', 'all_lysates', 'thesis-rerun', '1-graph', 'ram')
ram_files = [f for f in os.listdir(ram_dir) if '.npy' in f]

species2idx = {
    "Oleispira antarctica": 0,
    "Caenorhabditis elegans": 1,
    "Arabidopsis thaliana": 2,
    "Drosophila melanogaster": 3,
    "Danio rerio": 4,
    "Saccharomyces cerevisiae": 5,
    "Bacillus subtilis": 6,
    "Escherichia coli": 7,
    "Homo sapiens": 8,
    "Mus musculus": 9,
    "Geobacillus stearothermophilus": 10,
    "Picrophilus torridus": 11,
    "Thermus thermophilus": 12
}
cm = plt.get_cmap('gist_rainbow')

edge_types = ['pae', 'contact', 'backbone', 'codir', 'coord', 'deform']

In [ ]:
meta_file = osp.join(root_dir, 'data', 'collation', 'DeepSTABp', 'lysate.csv')

meta = np.loadtxt(meta_file, skiprows=1, delimiter=',', dtype=np.str_)
print(meta)

meta_accessions = meta[:,0]
meta_Tm = meta[:,1].astype(np.float_)
meta_ogts = meta[:,2].astype(np.float_)
meta_species = meta[:,3]

# accession_groups = []
# for ogt in np.unique(ogts):
#     accession_groups.append(accessions[ogts == ogt])

In [ ]:
test = False
save = True

In [ ]:
# sum statistics

sum_stats = np.empty((7, len(ram_files)))
accessions = []
species = []
tms = []

pbar = tqdm(ram_files)
for i, ram_file in enumerate(pbar):
    pbar.set_description(ram_file)
    if test and i > 5:
        break

    accession = ram_file.split('-')[0]
    accessions.append(accession)
    species.append(meta_species[meta_accessions==accession][0])
    tms.append(meta_Tm[meta_accessions==accession][0])

    # load data
    ram_path = os.path.join(ram_dir, ram_file)
    ram = np.load(ram_path)

    # remove negative values
    ram_abs = np.abs(ram)

    # sum RAM on all edge types
    ram_sum = np.vstack((ram_abs, np.sum(ram_abs, axis=0)))

    # sum of ramper residue
    sum_stats[:,i] = np.mean(ram_sum, axis=1)

accessions = np.array(accessions)
species = np.array(species)
tms = np.array(tms)

In [ ]:
norm_sumstats = sum_stats/sum_stats[[-1],:] * 100

avg = np.mean(norm_sumstats, axis=1)
std = np.std(norm_sumstats, axis=1)
max_val = np.max(norm_sumstats, axis=1)
min_val = np.min(norm_sumstats, axis=1)
mid = np.median(norm_sumstats, axis=1)

for i, edge_type in enumerate(edge_types + ['all']):
    print(f'{edge_type:8s}\t{avg[i]:>8.4f}\t{std[i]:>8.4f}\t{max_val[i]:>8.4f}\t{min_val[i]:>8.4f}\t{mid[i]:>8.4f}')

In [ ]:
# scatter: RAM percent vs Tm

fig, ax = plt.subplots(
    3, 2, sharex=True, sharey=True, figsize=(5.5,9), layout='compressed'
)
ax = ax.flatten()

pbar = tqdm(edge_types)
for i, edge_type in enumerate(pbar):
    pbar.set_description(edge_type)

    for s in np.unique(species):
        ax[i].scatter(
            norm_sumstats[i][species==s],
            tms[species==s],
            color=cm(species2idx[s]/len(species2idx)),
            s=1,
            alpha=0.5
        )
    
    ax[i].set_title(edge_type)

    ax[i].set_xlabel('contribution (%)')
    ax[i].tick_params(labelbottom=True)
    if i%2 == 0:
        ax[i].set_ylabel(r'$T_m$ (°C)')

if save:
    plt.savefig('RAM_percentage_vs_Tm.png', dpi=300)#, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
# scatter: RAM percent vs Tm

fig, ax = plt.subplots(
    2, 3, sharex=True, sharey=True, figsize=(8,6), layout='compressed'
)
ax = ax.flatten()

pbar = tqdm(['backbone', 'contact', 'pae', 'codir', 'coord', 'deform'])
for i, edge_type in enumerate(pbar):
    pbar.set_description(edge_type)

    for s in np.unique(species):
        ax[i].scatter(
            norm_sumstats[i][species==s],
            tms[species==s],
            color=cm(species2idx[s]/len(species2idx)),
            s=1,
            alpha=0.5
        )
    
    ax[i].set_title(edge_type)

    ax[i].set_xlabel('contribution (%)')
    ax[i].tick_params(labelbottom=True)
    if i%3 == 0:
        ax[i].set_ylabel(r'$T_m$ (°C)')

if save:
    plt.savefig('RAM_percentage_vs_Tm-horizontal.png', dpi=300)
plt.show()
plt.close()

In [ ]:
# histogram 

fig, ax = plt.subplots(
    3, 2, sharex=True, sharey=False, figsize=(5.5,9), layout='compressed'
)
ax = ax.flatten()

for i, edge_type in enumerate(['pae', 'contact', 'backbone', 'codir', 'coord', 'deform']):#, 'all']):

    ax[i].set_title(edge_type)

    ax[i].hist(norm_sumstats[i,:], bins=40, density=True)
    ax[i].set_xlabel('contribution (%)')
    ax[i].tick_params(labelbottom=True)
    if i%2 == 0:
        ax[i].set_ylabel('density')
    # ax[i].grid()

if save:
    plt.savefig('hist-RAM_contribution.png', dpi=300)
plt.show()
plt.close()

In [ ]:
# histogram 

fig, ax = plt.subplots(
    2, 3, sharex=True, sharey=False, figsize=(8,6), layout='compressed'
)
ax = ax.flatten()

for i, edge_type in enumerate(['backbone', 'contact', 'pae', 'codir', 'coord', 'deform']):

    ax[i].set_title(edge_type)

    ax[i].hist(norm_sumstats[i,:], bins=40, density=True)
    ax[i].set_xlabel('contribution (%)')
    ax[i].tick_params(labelbottom=True)
    if i%3 == 0:
        ax[i].set_ylabel('density')
    # ax[i].grid()

if save:
    plt.savefig('hist-RAM_contribution-horizontal.png', dpi=300)
plt.show()
plt.close()

In [ ]:
for i, ram_file in enumerate(os.listdir(ram_dir)):
    if test and i > 5:
        break

    print(ram_file)

    accession = ram_file.split('-')[0]

    ram_path = os.path.join(ram_dir, ram_file)
    ram = np.load(ram_path)

    fig, ax = plt.subplots(
        7, 1, sharex=True, sharey=True, figsize=(5.5, 9), layout='constrained' 
    )
    ax = ax.flatten()
    fig.suptitle(accession)

    n_res = ram.shape[1]
    res_idx = np.arange(n_res)

    y_abs_max = 0

    for i, edge_type in enumerate(['pae', 'contact', 'backbone', 'codir', 'coord', 'deform']):

        # ax[i].plot(np.where(ram[i]>0, ram[i], 0), linewidth=0.8)
        ax[i].bar(res_idx, np.where(ram[i]>0, ram[i], 0))

        ax[i].bar(res_idx, np.where(ram[i]>0, ram[i], 0), color='blue')
        ax[i].bar(res_idx, np.where(ram[i]<0, ram[i], 0), color='red')

        ax[i].axhline(y=0, c='gray', linewidth=0.5)

        ax[i].set_title(edge_type)

        ax[i].set_xlabel('residue')
        ax[i].set_ylabel('ram')
    
        # ax[i].set_xlim(0, ram[i].size)
        y_abs_max = max(y_abs_max, np.abs(ram[i]).max())

    i += 1
    ram_sum = np.sum(ram, axis=0)
    y_abs_max = max(y_abs_max, np.abs(ram_sum).max())

    # ax[i].plot(np.where(ram_sum>0, ram_sum, 0), linewidth=0.8)
    ax[i].bar(res_idx, np.where(ram_sum>0, ram_sum, 0))

    ax[i].bar(res_idx, np.where(ram_sum>0, ram_sum, 0), color='blue')
    ax[i].bar(res_idx, np.where(ram_sum<0, ram_sum, 0), color='red')

    ax[i].axhline(y=0, c='gray', linewidth=0.5)

    ax[i].set_title('Overall')

    ax[i].set_xlabel('residue')
    ax[i].set_ylabel('ram')

    ax[i].set_xlim(0, ram_sum.size)
    ax[i].set_ylim(-y_abs_max, y_abs_max)

    if save:
        plt.savefig(f'save/ram-{accession}.png', dpi=300)
    plt.show()
    plt.close()
    
    # break